In [211]:
import requests
import pandas as pd
import os



# NY Times
[In Dev - still need to refine)

In [32]:
search = 'Stuff'
filter = 'headline'
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={}&fq={}&api-key={}'.format(search,filter,api)
# request = requests.get(url)
# json = request.json()
# pd.json_normalize(json) #https://countwordsfree.com/jsonviewer
# json
url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Rishi&fq=headline&api-key=UD0AMQO6GY7mj3LEtk2EWGYMXMIKqI4r'

# Currents - News Aggregator API
[In Dev - still need to refine)

In [ ]:
# https://currentsapi.services/en/docs/search
apikey = api
category = "business"
language = "en" #languages['English']  # Mapping from Language to Code, e.g.: "English": "en"
country = "US" #regions["Canada"]  # Mapping from Country to Code, e.g.: "Canada": "CA",
keywords = "bitcoin"
t = "1"  # 1 for news, 2 for article and 3 for discussion content
domain = "financialpost.com"  # website primary domain name (without www or blog prefix)
start_date = "2021-01-01"  # YYYY-MM-DDTHH:MM:SS+00:00
query_url = f"https://api.currentsapi.services/v1/search?" \
            f"apiKey={apikey}" \
            f"&language={language}" \
            f"&category={category}" \
            f"&country={country}" \
            f"&type={t}" \
            f"&domain={domain}" \
            f"&keywords={keywords}" \
            f"&start_date={start_date}"

r = requests.get(query_url)
pprint(r.json())

# The Guardian API
[In Dev - still need to refine)

source: https://open-platform.theguardian.com/documentation/

In [229]:
url = "https://content.guardianapis.com/search?\
q=%22unilever%22&page=1&page-size=200&\
order-by=newest&from-date=2021-01-01&\
api-key={}".format(guardian_api)

def left(s, amount):
    return s[:amount]

def getTitles(url):
#     # get all News Titles
    response = requests.request("GET", url)
    a=response.json()['response']['results']
    results = []
    for result in a:
        results.append("{url}".format(url=result["webTitle"]))
#         results.append("{id}/{date} - {url}" .format(id=result["sectionName"], url=result["webTitle"], date=left(result["webPublicationDate"],10)))
    return results

# getTitles(url)

# response = requests.request("GET", url)
# a=response.json()['response']['results']
response_tbl = pd.json_normalize(a)
# response_tbl

In [ ]:
# create content
content = Content(api=guardian_api,headers='bitcoin')

# gets raw_response
raw_content = content.get_request_response()
print("Request Response status code {status}." .format(status=raw_content.status_code))
print("Request Response headers {header}." .format(header=raw_content.headers))

# content
print("Content Response headers {}." .format(content.response_headers()))

# get all results of a page
json_content = content.get_content_response()
all_results = content.get_results(json_content)
print("All results {}." .format(all_results))

# actual response
print("Response {response}" .format(response=json_content))

In [273]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia
sia = SIA()
results = []

searchterm = 'bitcoin'
fromdate = '2021-02-01'
url = "https://content.guardianapis.com/search?\
q=%22{}%22&page=1&page-size=200&\
order-by=newest&from-date={}&\
api-key={}".format(searchterm,fromdate,guardian_api)
print(url)
headlines = getTitles(url)

for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

# pprint(results[:3], width=100)
df = pd.DataFrame.from_records(results)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df[cols]

#Label Positive, Neutral & Negative News
df['label'] = 'Neutral'
df.loc[df['compound'] > 0, 'label'] = 'Positive'
df.loc[df['compound'] < 0, 'label'] = 'Negative'
df.head()

print("Positive headlines:\n")
pprint(list(df[df['label'] == 'Positive'].headline)[:5], width=200)


https://content.guardianapis.com/search?q=%22bitcoin%22&page=1&page-size=200&order-by=newest&from-date=2021-02-01&api-key=34ed4c23-c02c-49dd-a7cf-156d0433f156
Positive headlines:

['Readers reply: how can the economy grow year on year without limit – and why is this desirable?',
 "Bitcoin's market value exceeds $1tn after price soars",
 'Pound hits $1.40 on vaccine optimism; UK retail sales tumble 8.2% – as it happened',
 'Sound baths, self-help and teeth-grinding optimism: my strange, disorienting week on Clubhouse | Brigid Delaney',
 ' Barclays resumes dividend as 2020 profit falls 30%; markets await US data – business live']
